In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import joblib

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import spacy
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import statistics


import re
from os import path
from PIL import Image
import plotly
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.collocations import *
nltk.download('averaged_perceptron_tagger')

from textblob import TextBlob

import seaborn as sns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def normalize_list(data):
    x_min = min(data)
    x_max = max(data)
    if x_max-x_min == 0:
        return None
    return [(x-x_min)/(x_max-x_min) for x in data]

In [4]:
df = pd.read_csv('./home_depot_sentiments.csv', encoding='utf-8')

In [5]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,0,The Home Depot,5.0,1,more than 10 years,"You get out, what you put in",26-02-22,Department Supervisor,"Clarksville, TN",2,2,2,Probably one of the best retail places you cou...,"Expect normal workplace BS, but nothing too cr...","They’re decent, nothing extreme good or bad. I...",Probably one of the best retail places you cou...,0.00,0.63
1,1,The Home Depot,4.0,1,less than 1 year,Sales associate,26-02-22,Electrical Associate,NaN,0,0,0,To help all type of customer and making it eas...,They have some customers that get really angry...,They should have the night team to be more org...,To help all type of customer and making it eas...,0.00,0.77
2,2,The Home Depot,4.0,0,NaN,Not enough hours,26-02-22,Customer Service Associate,NaN,0,0,0,It was fun to work at the paint desk,Not enough hours when it was a lot of people w...,NaN,It was fun to work at the paint deskNot enough...,0.00,0.51
3,3,The Home Depot,4.0,0,more than 1 year,Worker,26-02-22,ISC,NaN,0,0,0,"WFH, not too many auto in calls",You get blamed for the field mistakes. You are...,NaN,"WFH, not too many auto in callsYou get blamed ...",0.00,-0.81
4,4,The Home Depot,4.0,0,more than 1 year,It was a good job great people,25-02-22,Freight Stocker,NaN,0,0,0,It was a good company to work,Just need a little bit of work on the inside,NaN,It was a good company to workJust need a littl...,0.79,0.44


In [6]:
df.columns

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee', 'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title', 'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros', 'Cons', 'Advice_to_Management', 'Pros_And_Cons', 'Review_Title_sentiment_score_vader', 'Pros_And_Cons_sentiment_score_vader'], dtype='object')

In [7]:
df.drop(['Unnamed: 0', 'Company'], axis=1, inplace=True)

In [8]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,1,more than 10 years,"You get out, what you put in",26-02-22,Department Supervisor,"Clarksville, TN",2,2,2,Probably one of the best retail places you cou...,"Expect normal workplace BS, but nothing too cr...","They’re decent, nothing extreme good or bad. I...",Probably one of the best retail places you cou...,0.00,0.63
1,4.0,1,less than 1 year,Sales associate,26-02-22,Electrical Associate,NaN,0,0,0,To help all type of customer and making it eas...,They have some customers that get really angry...,They should have the night team to be more org...,To help all type of customer and making it eas...,0.00,0.77
2,4.0,0,NaN,Not enough hours,26-02-22,Customer Service Associate,NaN,0,0,0,It was fun to work at the paint desk,Not enough hours when it was a lot of people w...,NaN,It was fun to work at the paint deskNot enough...,0.00,0.51
3,4.0,0,more than 1 year,Worker,26-02-22,ISC,NaN,0,0,0,"WFH, not too many auto in calls",You get blamed for the field mistakes. You are...,NaN,"WFH, not too many auto in callsYou get blamed ...",0.00,-0.81
4,4.0,0,more than 1 year,It was a good job great people,25-02-22,Freight Stocker,NaN,0,0,0,It was a good company to work,Just need a little bit of work on the inside,NaN,It was a good company to workJust need a littl...,0.79,0.44


In [9]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [10]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-01-01', freq='QS'))

In [11]:
len(quarters)

41

In [12]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [13]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,1,10,"You get out, what you put in",2022-02-26,Department Supervisor,"Clarksville, TN",2,2,2,Probably one of the best retail places you cou...,"Expect normal workplace BS, but nothing too cr...","They’re decent, nothing extreme good or bad. I...",Probably one of the best retail places you cou...,0.00,0.63
1,4.0,1,0,Sales associate,2022-02-26,Electrical Associate,NaN,0,0,0,To help all type of customer and making it eas...,They have some customers that get really angry...,They should have the night team to be more org...,To help all type of customer and making it eas...,0.00,0.77
2,4.0,0,NaN,Not enough hours,2022-02-26,Customer Service Associate,NaN,0,0,0,It was fun to work at the paint desk,Not enough hours when it was a lot of people w...,NaN,It was fun to work at the paint deskNot enough...,0.00,0.51
3,4.0,0,1,Worker,2022-02-26,ISC,NaN,0,0,0,"WFH, not too many auto in calls",You get blamed for the field mistakes. You are...,NaN,"WFH, not too many auto in callsYou get blamed ...",0.00,-0.81
4,4.0,0,1,It was a good job great people,2022-02-25,Freight Stocker,NaN,0,0,0,It was a good company to work,Just need a little bit of work on the inside,NaN,It was a good company to workJust need a littl...,0.79,0.44


In [14]:
df.drop('Job_Title', axis=1, inplace=True)

In [15]:
df.drop('Advice_to_Management', axis=1, inplace=True)

In [16]:
df.drop('Job_Location', axis=1, inplace=True)

In [17]:
df['Work_Duration'].fillna(df['Work_Duration'].mean(),inplace=True)

# other value aggregates

In [18]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()



# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [19]:

for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0


	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']


				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [20]:

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []


In [21]:
for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

<hr>

In [22]:
def clean_text(text):
	text=re.sub(r"\?", ".", text)
	text=re.sub(r"\!", ".", text)
	text=re.sub(r'([.])\1+', r'\1', text)
	rexp=r"\.(?=[A-Za-z]{1,15})"
	text=re.sub(rexp, ". ", text)
	return text

#this function extracts lemmas of nouns, verbs and adjectives
def lemma_words_pos_filtered(text):
    word_list=[]
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    for pos in pos_tokens:
        if (pos[1].startswith('N')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.NOUN)]
        if (pos[1].startswith('V')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.VERB)]
        if (pos[1].startswith('J')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.ADJ)]
    word_list=[word for word in word_list if word not in  
                                    stopwords.words('english') ]
    return  " ".join(word_list)

#to keep only unique tuples
def unique_tuples(list_of_tuples):
     list_ = [tuple(sorted(t)) for t in list_of_tuples]
     list_ = list(set(list_))
     return list_

def findbigramsintext(text):
	mylist=nltk.word_tokenize(text)
	list1=[x for x in mylist]
	feature_list = []
	for i in range(length_features2):
	# In each iteration, add an empty list to the main list      
			feature_list.append([])
	i=0
	# bigram_features is the list of unique set of bigrams found after collocation
	for l in bigram_features:
			list2=[x for x in l]
			result =  all(elem in list1  for elem in list2)
			if result: 
				feature_list[i].append(' '.join(list2))
			i=i+1
	return feature_list


def keepnonempty(list1):
       mylist= [x for x in list1 if x != []]
       return mylist
def flatten_list(row_list):
        l = [item for inner_list in row_list for item in inner_list]
        return l


In [23]:
import nltk
nltk.download('punkt')
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/sameer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sameer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# text cleaning

In [24]:

df['Cleaned_Pros_and_Cons_Text'] = df['Pros_And_Cons'].apply(sent_tokenize)

In [25]:
len(df)

34720

In [26]:
df = df.explode('Cleaned_Pros_and_Cons_Text', ignore_index=False)

In [27]:
len(df)

75812

In [28]:
df['lemma_words_pos_filtered'] = df['Cleaned_Pros_and_Cons_Text'].apply(lemma_words_pos_filtered)

In [29]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words( " ".join(df['lemma_words_pos_filtered']).split(),window_size=5)
#keeping bigrams that occur at least 5 times
finder.apply_freq_filter(5)
bigram_list_tuples=finder.nbest(bigram_measures.likelihood_ratio, 20)

In [30]:
bigram_list_tuples

[('home', 'depot'),
 ('full', 'time'),
 ('part', 'time'),
 ('life', 'balance'),
 ('customer', 'service'),
 ('co', 'worker'),
 ('flexible', 'schedule'),
 ('take', 'care'),
 ('credit', 'card'),
 ('tuition', 'reimbursement'),
 ('minimum', 'wage'),
 ('none', 'none'),
 ('profit', 'sharing'),
 ('success', 'sharing'),
 ('flexible', 'hour'),
 ('*', '*'),
 ('success', 'share'),
 ('place', 'work'),
 ('work/life', 'balance'),
 ('hour', 'week')]

In [31]:
bigram_features=unique_tuples(bigram_list_tuples)
length_features2=len(bigram_features)

In [32]:
df['bigram_list'] = df['lemma_words_pos_filtered'].apply(findbigramsintext)

In [33]:
df['bigrams']=df['bigram_list'].apply(keepnonempty)
df['bigrams']=df['bigrams'].apply(flatten_list)

In [34]:
#Sentiment scores are calculated using libraries TextBlob and Vader
df['polarity'] = df['Cleaned_Pros_and_Cons_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [35]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader,Cleaned_Pros_and_Cons_Text,lemma_words_pos_filtered,bigram_list,bigrams,polarity
0,5.0,1,10.000000,"You get out, what you put in",2022-02-26,2,2,2,Probably one of the best retail places you cou...,"Expect normal workplace BS, but nothing too cr...",Probably one of the best retail places you cou...,0.0,0.63,Probably one of the best retail places you cou...,best retail place workexpect normal workplace ...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.183333
1,4.0,1,0.000000,Sales associate,2022-02-26,0,0,0,To help all type of customer and making it eas...,They have some customers that get really angry...,To help all type of customer and making it eas...,0.0,0.77,To help all type of customer and making it eas...,help type customer make easy shop,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.000000
1,4.0,1,0.000000,Sales associate,2022-02-26,0,0,0,To help all type of customer and making it eas...,They have some customers that get really angry...,To help all type of customer and making it eas...,0.0,0.77,Also I have a great time because everyone ther...,great time everyone nice peoplethey customer g...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.300000
2,4.0,0,2.359233,Not enough hours,2022-02-26,0,0,0,It was fun to work at the paint desk,Not enough hours when it was a lot of people w...,It was fun to work at the paint deskNot enough...,0.0,0.51,It was fun to work at the paint deskNot enough...,fun work paint desknot enough hour lot people ...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.150000
3,4.0,0,1.000000,Worker,2022-02-26,0,0,0,"WFH, not too many auto in calls",You get blamed for the field mistakes. You are...,"WFH, not too many auto in callsYou get blamed ...",0.0,-0.81,"WFH, not too many auto in callsYou get blamed ...",wfh many auto callsyou get blame field mistake,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.500000


In [36]:
copy_df = df.copy()

In [37]:
copy_df = copy_df.explode('bigrams', ignore_index=False)

In [38]:
res = copy_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

In [39]:
print(res)

                  bigrams  polarity
0                     * *  0.090206
1            balance life  0.230800
2       balance work/life  0.129895
3             card credit  0.067139
4               care take  0.184167
5               co worker  0.291346
6        customer service  0.235922
7              depot home  0.214410
8           flexible hour  0.216959
9       flexible schedule  0.198583
10              full time  0.252448
11              hour week  0.106962
12           minimum wage  0.150231
13              none none  0.159682
14              part time  0.205915
15             place work  0.399449
16         profit sharing  0.160210
17  reimbursement tuition  0.192226
18          share success  0.202114
19        sharing success  0.236495


In [40]:
all_bigrams = res['bigrams'].tolist()
print(all_bigrams)

['* *', 'balance life', 'balance work/life', 'card credit', 'care take', 'co worker', 'customer service', 'depot home', 'flexible hour', 'flexible schedule', 'full time', 'hour week', 'minimum wage', 'none none', 'part time', 'place work', 'profit sharing', 'reimbursement tuition', 'share success', 'sharing success']


In [41]:
result = copy_df[['bigrams', 'polarity']].groupby('bigrams')['polarity'].mean()

In [42]:
topic_polarities = dict()
for b in all_bigrams:
	topic_polarities[b] = []

In [43]:
polarity_results = []
for i in range(len(quarters)):
	if i < len(quarters) - 1:
		qtr_start = quarters[i]
		qtr_end = quarters[i+1]
		cur_df = df[(df['Date_Posted'] >= qtr_start) & (df['Date_Posted'] < qtr_end)]
		if(len(cur_df) == 0):
			continue
		cur_df = cur_df.explode('bigrams', ignore_index=False)
		results = cur_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

		for bigram in all_bigrams:
			if bigram in results['bigrams'].tolist():
				polarity = results[results['bigrams'] == bigram]['polarity'].tolist()[0]
				topic_polarities[bigram].append(polarity)
				# polarity_results.append(results[results['bigrams'] == bigram]['polarity'].tolist()[0])
			else:
				topic_polarities[bigram].append(None)

		# for i, row in results.iterrows():
		# 	if row['bigrams'] in topic_polarities:
		# 		topic_polarities[row['bigrams']].append(row['polarity'])
		# 	else:
		# 		topic_polarities[row['bigrams']] = [row['polarity']]
		polarity_results.append(results)


In [44]:
len(topic_polarities)

20

In [45]:
len(polarity_results)

40

In [46]:
for key, value in topic_polarities.items():
	print(key, len(value))

* * 40
balance life 40
balance work/life 40
card credit 40
care take 40
co worker 40
customer service 40
depot home 40
flexible hour 40
flexible schedule 40
full time 40
hour week 40
minimum wage 40
none none 40
part time 40
place work 40
profit sharing 40
reimbursement tuition 40
share success 40
sharing success 40


In [47]:
print(counts)

{'Q1': 130, 'Q2': 197, 'Q3': 172, 'Q4': 155, 'Q5': 191, 'Q6': 153, 'Q7': 258, 'Q8': 233, 'Q9': 308, 'Q10': 357, 'Q11': 351, 'Q12': 336, 'Q13': 458, 'Q14': 673, 'Q15': 791, 'Q16': 578, 'Q17': 1166, 'Q18': 1162, 'Q19': 1528, 'Q20': 749, 'Q21': 778, 'Q22': 614, 'Q23': 1058, 'Q24': 1285, 'Q25': 1109, 'Q26': 1030, 'Q27': 872, 'Q28': 669, 'Q29': 1019, 'Q30': 986, 'Q31': 779, 'Q32': 360, 'Q33': 501, 'Q34': 430, 'Q35': 938, 'Q36': 1700, 'Q37': 2369, 'Q38': 2077, 'Q39': 1992, 'Q40': 1810, 'Q41': 0}


In [48]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [49]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,None,0.650148,0.000000,0.097335,1.000000,0.000000
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,None,0.475303,0.367620,0.506843,0.800532,0.224598
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,None,0.122596,1.000000,0.593825,0.698052,0.549565
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,None,0.733251,0.803056,0.700692,0.769748,0.804469
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,None,0.458820,0.675393,0.795683,0.816844,0.905554
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,None,0.635181,0.783970,0.983947,0.676106,0.841275
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,None,0.663462,0.728070,0.403431,0.637172,0.810705
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,None,0.955637,0.728484,0.377487,0.554365,0.733690
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,None,0.684534,0.749487,0.542363,0.629083,0.777044
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,None,0.826277,0.786083,0.582626,0.592624,0.681658


In [50]:
all_bigrams

['* *',
 'balance life',
 'balance work/life',
 'card credit',
 'care take',
 'co worker',
 'customer service',
 'depot home',
 'flexible hour',
 'flexible schedule',
 'full time',
 'hour week',
 'minimum wage',
 'none none',
 'part time',
 'place work',
 'profit sharing',
 'reimbursement tuition',
 'share success',
 'sharing success']

In [51]:
for key,value in topic_polarities.items():
	col_name = key.replace(' ', '_')
	final[col_name] = value

In [52]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,None,0.650148,0.000000,0.097335,1.000000,0.000000,-0.144949,0.700000,0.087698,NaN,0.121660,NaN,0.002473,0.170992,0.194545,0.000000,0.261838,0.033333,-0.439583,NaN,0.174028,0.245952,0.541667,NaN,0.325000,0.300000
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,None,0.475303,0.367620,0.506843,0.800532,0.224598,NaN,0.069444,0.273951,NaN,0.098810,NaN,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,NaN,0.248525,0.378884,NaN,0.083333,-0.086458,0.333984
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,None,0.122596,1.000000,0.593825,0.698052,0.549565,NaN,0.072667,0.500000,-1.458333e-01,0.123791,NaN,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,NaN,0.141372,0.141372
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,None,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,NaN,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,NaN,NaN,0.052771,0.358333,NaN,NaN,-0.166667,0.154911
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,None,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,NaN,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,NaN,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,None,0.635181,0.783970,0.983947,0.676106,0.841275,NaN,0.000000,NaN,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,NaN,0.060000,0.283333
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,None,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,None,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,NaN,-0.105724,0.105303
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,None,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,NaN,0.236111,0.338889
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,None,0.826277,0.786083,0.582626,0.592624,0.681658,NaN,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.220661,-0.164242,0.036111,0.114323,0.392188


In [53]:
for key, value in topic_polarities.items():
	col = key.replace(' ', '_')
	final[col].fillna(final[col].mean(),inplace=True)

In [54]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,None,0.650148,0.000000,0.097335,1.000000,0.000000,-0.144949,0.700000,0.087698,5.930070e-02,0.121660,0.318611,0.002473,0.170992,0.194545,0.000000,0.261838,0.033333,-0.439583,0.182922,0.174028,0.245952,0.541667,0.205315,0.325000,0.300000
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,None,0.475303,0.367620,0.506843,0.800532,0.224598,0.147775,0.069444,0.273951,5.930070e-02,0.098810,0.318611,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,0.182922,0.248525,0.378884,0.198719,0.083333,-0.086458,0.333984
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,None,0.122596,1.000000,0.593825,0.698052,0.549565,0.147775,0.072667,0.500000,-1.458333e-01,0.123791,0.318611,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,0.205315,0.141372,0.141372
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,None,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,0.318611,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,0.122382,0.182922,0.052771,0.358333,0.198719,0.205315,-0.166667,0.154911
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,None,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,5.930070e-02,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,0.122382,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,None,0.635181,0.783970,0.983947,0.676106,0.841275,0.147775,0.000000,0.137187,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,0.205315,0.060000,0.283333
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,None,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,None,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,0.205315,-0.105724,0.105303
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,None,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,0.205315,0.236111,0.338889
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,None,0.826277,0.786083,0.582626,0.592624,0.681658,0.147775,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.220661,-0.164242,0.036111,0.114323,0.392188


In [55]:
len(final)

40

In [56]:
earnings = [
4.129,
4.807,
4.145,
2.857,
3.432,
4.332,
2.245,
2.481,
2.769,
3.479,
2.513,
2.344,
2.867,
3.506,
2.404,
1.779,
2.165,
2.672,
2.014,
1.744,
1.969,
2.441,
1.803,
1.471,
1.725,
2.234,
1.579,
1.379,
1.537,
2.05,
1.379,
1.013,
1.351,
1.795,
1.226,
1.021,
0.947,
1.532,
1.035,
0.774
]

earnings = earnings[::-1]

In [58]:
len(earnings)

40

In [59]:
final['earnings'] = earnings

In [60]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success,earnings
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,None,0.650148,0.000000,0.097335,1.000000,0.000000,-0.144949,0.700000,0.087698,5.930070e-02,0.121660,0.318611,0.002473,0.170992,0.194545,0.000000,0.261838,0.033333,-0.439583,0.182922,0.174028,0.245952,0.541667,0.205315,0.325000,0.300000,0.774
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,None,0.475303,0.367620,0.506843,0.800532,0.224598,0.147775,0.069444,0.273951,5.930070e-02,0.098810,0.318611,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,0.182922,0.248525,0.378884,0.198719,0.083333,-0.086458,0.333984,1.035
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,None,0.122596,1.000000,0.593825,0.698052,0.549565,0.147775,0.072667,0.500000,-1.458333e-01,0.123791,0.318611,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,0.205315,0.141372,0.141372,1.532
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,None,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,0.318611,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,0.122382,0.182922,0.052771,0.358333,0.198719,0.205315,-0.166667,0.154911,0.947
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,None,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,5.930070e-02,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,0.122382,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042,1.021
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,None,0.635181,0.783970,0.983947,0.676106,0.841275,0.147775,0.000000,0.137187,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,0.205315,0.060000,0.283333,1.226
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,None,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444,1.795
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,None,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,0.205315,-0.105724,0.105303,1.351
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,None,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,0.205315,0.236111,0.338889,1.013
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,None,0.826277,0.786083,0.582626,0.592624,0.681658,0.147775,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.220661,-0.164242,0.036111,0.114323,0.392188,1.379


In [61]:
pred_column = [None]
for i in range(1, len(earnings)):
	if earnings[i] > earnings[i-1]:
		pred_column.append(1)
	else:
		pred_column.append(0)

In [62]:
pred_column

[None,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0]

In [63]:
final['pred_column'] = pred_column

In [64]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success,earnings,pred_column
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,None,0.650148,0.000000,0.097335,1.000000,0.000000,-0.144949,0.700000,0.087698,5.930070e-02,0.121660,0.318611,0.002473,0.170992,0.194545,0.000000,0.261838,0.033333,-0.439583,0.182922,0.174028,0.245952,0.541667,0.205315,0.325000,0.300000,0.774,NaN
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,None,0.475303,0.367620,0.506843,0.800532,0.224598,0.147775,0.069444,0.273951,5.930070e-02,0.098810,0.318611,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,0.182922,0.248525,0.378884,0.198719,0.083333,-0.086458,0.333984,1.035,1.0
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,None,0.122596,1.000000,0.593825,0.698052,0.549565,0.147775,0.072667,0.500000,-1.458333e-01,0.123791,0.318611,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,0.205315,0.141372,0.141372,1.532,1.0
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,None,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,0.318611,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,0.122382,0.182922,0.052771,0.358333,0.198719,0.205315,-0.166667,0.154911,0.947,0.0
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,None,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,5.930070e-02,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,0.122382,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042,1.021,1.0
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,None,0.635181,0.783970,0.983947,0.676106,0.841275,0.147775,0.000000,0.137187,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,0.205315,0.060000,0.283333,1.226,1.0
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,None,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444,1.795,1.0
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,None,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,0.205315,-0.105724,0.105303,1.351,0.0
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,None,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,0.205315,0.236111,0.338889,1.013,0.0
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,None,0.826277,0.786083,0.582626,0.592624,0.681658,0.147775,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.22066

In [65]:
final.drop(['recommend_same'], axis=1, inplace=True)

In [66]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success,earnings,pred_column
0,Q1,0.357823,0.694583,0.960191,0.326643,1,2,2,0,0.180013,0.254717,0.000000,0.984208,0.000000,1.000000,0.650148,0.000000,0.097335,1.000000,0.000000,-0.144949,0.700000,0.087698,5.930070e-02,0.121660,0.318611,0.002473,0.170992,0.194545,0.000000,0.261838,0.033333,-0.439583,0.182922,0.174028,0.245952,0.541667,0.205315,0.325000,0.300000,0.774,NaN
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.394778,0.103374,0.642293,0.475303,0.367620,0.506843,0.800532,0.224598,0.147775,0.069444,0.273951,5.930070e-02,0.098810,0.318611,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,0.182922,0.248525,0.378884,0.198719,0.083333,-0.086458,0.333984,1.035,1.0
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.246961,0.268684,0.068264,0.122596,1.000000,0.593825,0.698052,0.549565,0.147775,0.072667,0.500000,-1.458333e-01,0.123791,0.318611,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,0.205315,0.141372,0.141372,1.532,1.0
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.054827,0.184796,0.032069,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,0.318611,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,0.122382,0.182922,0.052771,0.358333,0.198719,0.205315,-0.166667,0.154911,0.947,0.0
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.149871,0.167156,0.018133,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,5.930070e-02,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,0.122382,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042,1.021,1.0
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.099837,0.261868,0.071417,0.635181,0.783970,0.983947,0.676106,0.841275,0.147775,0.000000,0.137187,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,0.205315,0.060000,0.283333,1.226,1.0
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.100077,0.229082,0.013197,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444,1.795,1.0
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.000000,0.159963,0.000000,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,0.205315,-0.105724,0.105303,1.351,0.0
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.241239,0.289428,0.047925,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,0.205315,0.236111,0.338889,1.013,0.0
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.220732,0.263988,0.054437,0.826277,0.786083,0.582626,0.592624,0.681658,0.147775,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.220661,-0.164242,0.036111,0.114323,0.392188,1.379,1.0


In [67]:
final.dropna(inplace=True)

In [68]:
final.drop(['recommend_unknown', 'ceo_unknown', 'business_unknown'], axis=1, inplace=True)

In [69]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,balance_life,balance_work/life,card_credit,care_take,co_worker,customer_service,depot_home,flexible_hour,flexible_schedule,full_time,hour_week,minimum_wage,none_none,part_time,place_work,profit_sharing,reimbursement_tuition,share_success,sharing_success,earnings,pred_column
1,Q2,0.347274,0.841432,1.000000,0.490944,1,2,2,0,0.549748,0.784803,0.210265,0.475303,0.367620,0.506843,0.800532,0.224598,0.147775,0.069444,0.273951,5.930070e-02,0.098810,0.318611,0.137472,0.285668,0.116667,0.300000,0.234524,0.051847,0.305401,0.182922,0.248525,0.378884,0.198719,0.083333,-0.086458,0.333984,1.035,1.0
2,Q3,0.112603,0.537575,0.206508,0.906557,1,2,2,1,0.650358,1.000000,0.541860,0.122596,1.000000,0.593825,0.698052,0.549565,0.147775,0.072667,0.500000,-1.458333e-01,0.123791,0.318611,0.396386,0.069635,0.550000,0.187847,0.212870,-0.028367,-0.166667,0.233333,0.177840,0.275583,0.141372,0.205315,0.141372,0.141372,1.532,1.0
3,Q4,0.119444,0.504294,0.349575,0.775782,1,2,2,2,0.784241,0.258751,0.501075,0.733251,0.803056,0.700692,0.769748,0.804469,0.155833,0.095170,0.206463,-2.218750e-01,-0.031250,0.318611,0.285341,0.072905,0.007917,0.069583,0.214950,0.131373,0.122382,0.182922,0.052771,0.358333,0.198719,0.205315,-0.166667,0.154911,0.947,0.0
4,Q5,0.223843,0.494634,0.615800,1.000000,1,2,2,2,0.643790,0.523175,0.596393,0.458820,0.675393,0.795683,0.816844,0.905554,0.166667,0.000000,0.380858,5.930070e-02,0.171875,0.433333,0.051042,0.182844,0.358333,0.174065,0.229885,0.055666,0.122382,0.325000,-0.066984,0.159259,0.700000,0.066667,0.084127,0.126042,1.021,1.0
5,Q6,0.075728,0.488340,0.415072,0.901418,1,2,2,2,0.598119,0.426371,0.270733,0.635181,0.783970,0.983947,0.676106,0.841275,0.147775,0.000000,0.137187,-2.583333e-01,0.087500,0.500000,0.433333,0.228253,0.262446,0.000000,0.098186,-0.016105,0.395833,-0.065000,0.167163,0.139120,0.075534,0.205315,0.060000,0.283333,1.226,1.0
6,Q7,0.099373,0.414912,0.000000,0.696889,1,2,2,2,0.889298,0.637659,0.742550,0.663462,0.728070,0.403431,0.637172,0.810705,0.296653,0.086667,0.560000,0.000000e+00,-0.043750,0.250000,0.274362,0.076017,0.457500,0.355225,0.182571,0.256037,0.068615,0.200000,0.070953,0.442603,-0.024365,0.140197,0.016667,0.194444,1.795,1.0
7,Q8,0.000000,0.279331,0.142526,0.605071,1,2,2,2,1.000000,0.780729,1.000000,0.955637,0.728484,0.377487,0.554365,0.733690,0.167708,0.225000,0.525000,-1.387779e-17,0.377778,0.430280,0.253172,0.165550,-0.042273,0.221608,0.235903,0.021875,0.058958,0.016667,0.199650,0.363021,0.152803,0.205315,-0.105724,0.105303,1.351,0.0
8,Q9,0.154726,0.619323,0.428092,0.816669,1,2,2,2,0.681777,0.421196,0.622006,0.684534,0.749487,0.542363,0.629083,0.777044,0.267361,0.237500,-0.202083,-6.111111e-02,0.103861,0.415751,0.235384,0.172252,0.166315,0.103480,0.246302,0.035456,0.135011,0.200000,0.224134,0.302334,0.151042,0.205315,0.236111,0.338889,1.013,0.0
9,Q10,0.063889,0.528016,0.434145,0.832159,1,2,2,1,0.681581,0.449141,0.754186,0.826277,0.786083,0.582626,0.592624,0.681658,0.147775,0.037856,-0.225189,-2.857143e-02,0.169643,0.209286,0.179969,0.149775,0.137045,0.059815,0.239437,-0.001858,0.033191,0.392857,0.178316,0.220661,-0.164242,0.036111,0.114323,0.392188,1.379,1.0
10,Q11,0.094688,0.589237,0.726878,0.622674,1,2,2,1,0.708663,0.532633,0.649066,1.000000,0.799520,0.198798,0.412066,0.561524,0.147775,0.205179,-0.066786,0.000000e+00,0.306667,0.306296,0.228093,0.180703,0.147222,0.178287,0.253077,0.006845,0.281953,0.095326,0.216002,0.280544,0.056250,0.229167,0.136111,0.146514,2.050,1.0


In [71]:
final.to_excel('home_depot_cleaned_data.xlsx')

In [72]:
final.to_csv('home_depot_cleaned_data.csv')